### **SEXY PANDAS**
#### An attempt to restyle Pandas Data Frames to reflect Excel-style formatting, and to make re-use of the functions easy

#### Currently works to style using a customizable dicationary config.  Next steps are:
- Making an expanded set of pre-configured styles, for super-fast operation
- Moving from function-based to class-inheritance based, directly extending pandas
----

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import json
from collections import defaultdict

In [7]:
import dataframe_image # use this package to help print out our new styles

#### **GLOBAL VARIABLES**

In [8]:
# set of predefined RGB colors
PALETTES_RGB = {
    'yellow':[252,239,166],
    'green':[122,188,129],
    'red':[231,114,112],
    'white':[255,255,255],
    'blue':[101,147,194],
    'grey':[144,144,148],
    'sns_blue':[13,29,85],
    'sns_yellow':[255,255,221],
    'sns_green':[103,182,193]
}
PALETTES_HEX = {}

#.hide_index()

#cols=[[['sns_yellow','sns_green','sns_blue'],'shade']],
#[['sns_blue','sns_green'],'shade',['A','C']]]

#).format('{:.3f}').hide_index()

#.highlight_max(color='lightgreen')
#.bar(subset='C',color='#AAC')

#### **HELPER FUNCTIONS**

In [9]:
# self-ingestion to get out our python code when regular export fails
def get_raw_python_from_notebook(notebook,python=None):
    if python is None: python=notebook
    with open(notebook+'.ipynb','r') as f:
        rawpy = json.load(f)
    rawpy = [[] if c['source'] == [] else c['source'] for c in rawpy['cells'] if c['cell_type']=='code']
    for r in rawpy:
        r.extend(['\n','\n'])
    raw = [l for r in rawpy for l in r]
    with open(python+'.py', 'w') as f:
        f.write(''.join(raw))
get_raw_python_from_notebook('main')

In [10]:
# extract the hex value from a given color and round accordingly, ensuring length==2
def make_hex_color(s, round='nearest'):
    if round=='up':
        s_round = np.ceil(s,0)
    elif round=='down':
        s_round = np.floor(s,0)
    else:
        s_round = np.round(s,0)
    return ('0'+hex(int(s_round))[2:].upper())[-2:]

In [11]:
# make a full hex color from 3 RGB channels
def rgb_to_hex(channels, round='nearest'):
    return '#'+(''.join([make_hex_color(c, round) for c in channels]))

In [12]:
# use our helpers to populate hex code dict (we will want to speak generally in hex for this work)
for i in PALETTES_RGB.keys():
    PALETTES_HEX[i] = '#'+(''.join([make_hex_color(color) for color in PALETTES_RGB[i]]))

In [13]:
# make a range of evenly spaced floats of a given min, max and length
def divide_range(mymin, mymax, size, thresholds):
    return [mymin+(k*(mymax-mymin)/(size-1)) for k in range(size)]
    # np.arange(mymin,mymax+(1/(size-1)),(1/(size-1))) # alternative way

In [14]:
def make_quantiles(values, n, mn, mx, spacing='relative'):
    if type(n)==list: n=len(n)
    if spacing == 'even':  # evenly distribute the color palette ignoring the magnitude of the values
        return [np.floor((n-1)*((values<=v).mean()+(values<v).mean())/2) for v in values]
    elif spacing == 'relative':  # factor in the megnitude of the values when making the visuals (default)
        return [np.maximum(0,np.minimum(int((n-1)*(v-mn)/(mx-mn)),n-2)) for v in values] # prevent negative values

In [15]:
# get RGB colors from hex if we want to go the other way
def get_rgb_colors(c):
    if c in PALETTES_RGB:
        return PALETTES_RGB[c]
    else:
        c = c.replace('#','')
        n = [c[i*int(len(c)/3):(i+1)*int(len(c)/3)] for i in range(3)]
        if len(c)==3: n = [s+s for s in n]
        return [int(n,16) for n in n]

In [16]:
# generates an RGB color value from a given float, based on its distance from defined min/max values and their associated rgb colors
def generate_color(value, thresholds, colors):
    (min,max) = thresholds
    (min_color,max_color) = colors
    diff = [min_color[i]-max_color[i] for i in range(3)]
    return [min_color[j]-(diff[j]*(value-min)/(max-min)) for j in range(3)]

In [17]:
def luminosity(v):
    return (0.2126*v[0]+0.7152*v[1]+0.0722*v[2])

In [18]:
def type_format(data,val,number):
    if number in [None,'abs']:
        return max(np.min(data),min(np.max(data),val))
    elif number=='pct':
        return np.quantile(data,val/100)

In [151]:
def run_transform(data,funcs):
    for f in funcs:
        if f==np.log:
            data = np.abs(data)
        try:
            data = f(data)
        except:
            data = f(np.abs(data))
        data = data.replace(np.nan, 0)
    return data

#### **MAIN STYLING FUNCTIONS**

In [31]:
def apply_colors_legacy(col, palette=['yellow', 'green'], default_fill_color='#FFF', default_text_color='#000', type='shade', rows=None, columns=None, mymin=None, mymax=None):
    # by default, use column-wise min and max if nothing is provided
    if mymax is None: mymin, mymax = min(col.values), max(col.values)
    
    # to prevent a divide by zero later on - max must always be greater than min
    if mymax==mymin: mymax=mymin+1
    palette = [get_rgb_colors(p) for p in palette]
    
    if len(palette) == 1:
        # if the palette length is just 1 we just apply it globally - the trivial case
        rgb_vals = [palette[0] for c in col.values]
    else:
        # if the palette length is greater than 1, we assign each value a bespoke color based on its position in the full range
        thresholds = divide_range(mymin, mymax, len(palette))
        quantiles = make_quantiles(col.values, palette, mymin, mymax)
        rgb_vals = [generate_color(c, thresholds[q:q+2], palette[q:q+2]) for c,q in zip(col.values, quantiles)]

    def filter_cells(inputs, default=''):  
        if (columns is not None):
            inputs = [inputs[j] if (mymin <= col.values[j] <= mymax) and (col.name in columns) else default for j in range(len(col.values))]
        if (rows is not None):
            inputs = [inputs[j] if (mymin <= col.values[j] <= mymax) and (j in rows) else default for j in range(len(col.values))]
        return inputs
    
    if type == 'shade':
        res = ['background-color: #'+(''.join([make_hex_color(c) for c in v])) for v in rgb_vals]
        default = 'background-color: '+default_fill_color
        return filter_cells(res, default)
    elif type == 'text_shade':
        tx = ['color: '+('#000' if luminosity(v)>=100 else '#FFF') for v in rgb_vals]
        default = 'color: '+default_text_color
        return filter_cells(tx, default)
    else:
        return ['' for c in col.values]

In [32]:
def pretty_pandas_legacy(df, fill_palette=['yellow','green'], rows=None, columns=None, index='show', group=None, font_size=None, header_size=None,
                  default_fill_color='#FFF', default_text_color='#000', bg='white', mymin=None, mymax=None):
    """Generate efficient dataframe styling with fully customizable inputs.

    Keyword arguments:
    todo
    """
    sdf = df.style
    rows_all,columns_all = list(df.index),list(df.columns)
    if mymin is None: mymin=np.min(df.values)
    if mymax is None: mymax=np.max(df.values)

    if index=='hide': sdf.hide_index()
    if header_size is None: header_size=font_size
    if type(fill_palette[0]) != list: fill_palette=[fill_palette] 
    
    for palette in fill_palette:
        row_subset = rows_all if rows is None else [r for r in rows if r in rows_all]
        row_index_subset = [rows_all.index(r) for r in row_subset]
        col_subset = columns_all if columns is None else [c for c in columns if c in columns_all]
        d = df.loc[row_subset,col_subset]
        mymin = max(mymin, np.min(d.values)) if group is None else None
        mymax = min(mymax, np.max(d.values)) if group is None else None
        sdf.apply(apply_colors_legacy, palette=palette, default_fill_color=default_fill_color, default_text_color=default_text_color,
                  type='shade', rows=row_index_subset, columns=col_subset, mymin=mymin, mymax=mymax, axis=0)
        sdf.apply(apply_colors_legacy, palette=palette, default_fill_color=default_fill_color, default_text_color=default_text_color,
                  type='text_shade', rows=row_index_subset, columns=col_subset, mymin=mymin, mymax=mymax, axis=0)

    return sdf.format('{:.3f}').set_table_styles([{'selector':'tr','props':[('background-color',bg+' !important')]}])

    # sdf.set_properties(**{'font-size': str(font_size)+'pt'})
    # .set_table_styles([{'selector': 'th', 'props': [('font-size', str(22)+'pt !important')]}])

#####
The **pretty_pandas** function acts as the main entry point to the styling, and can work with text and fill colors and other formatting:

#### **EXAMPLES**

In [23]:
alpha = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
test_df = pd.DataFrame([np.arange(26)+(2*np.random.random(26)) for i in range(26)],columns=[a for a in alpha][:26])
# test_df = np.round(test_df,2)

fruits = ['Apple','Watermelon','Orange','Pear','Cherry','Strawberry','Nectarine','Grape','Mango','Blueberry','Pomegranate','Starfruit','Plum','Banana',
          'Raspberry','Mandarin','Jackfruit','Papaya','Kiwi','Pineapple','Lime','Lemon','Apricot','Grapefruit','Melon','Coconut','Avocado','Peach']

test_df.index = fruits[:26]
np.random.shuffle(fruits)

In [21]:
pretty_pandas(test_df)

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z
Apple,1.322,1.637,3.672,3.688,5.272,5.750,7.064,8.777,9.274,10.732,11.873,12.679,12.424,13.487,14.277,15.436,16.074,18.944,18.129,20.546,20.894,22.147,22.785,24.366,24.497,26.824
Watermelon,0.681,2.885,3.863,4.230,4.710,6.214,7.784,8.963,9.173,10.464,11.931,12.225,12.353,14.942,14.930,15.897,16.566,18.433,19.911,19.974,20.694,21.521,22.071,23.821,24.136,26.591
Orange,1.242,2.314,3.936,4.028,4.849,6.944,6.956,7.332,9.971,9.311,10.397,12.182,12.043,14.284,14.191,15.858,17.744,18.702,18.951,19.080,20.718,21.468,22.075,24.437,25.763,26.952
Pear,0.986,1.171,3.159,4.820,5.849,6.030,6.239,8.239,8.447,10.519,10.541,11.342,13.057,13.647,15.201,15.356,16.774,17.345,18.670,20.224,20.044,22.628,23.433,23.923,25.307,26.027
Cherry,1.924,1.361,2.048,3.689,5.707,6.540,6.722,8.137,9.772,9.836,11.821,11.095,13.525,13.951,15.742,16.395,17.629,17.478,19.367,19.734,21.380,22.088,22.664,24.296,24.482,25.108
Strawberry,0.617,2.142,2.982,4.194,4.495,6.697,7.243,7.118,8.407,10.260,11.691,12.120,12.932,14.655,14.715,15.897,16.983,18.431,19.294,19.023,20.267,21.907,22.302,23.359,25.474,25.886
Nectarine,1.989,1.451,2.180,4.265,4.319,5.025,7.734,8.041,9.729,9.408,10.612,12.553,13.633,14.161,14.978,16.980,16.386,18.994,18.460,20.574,20.654,22.548,22.595,24.533,24.030,25.555
Grape,0.689,2.889,3.051,3.277,4.792,6.272,7.368,7.709,8.084,10.763,11.474,11.282,13.442,14.644,15.638,15.118,16.074,17.463,19.343,19.159,20.258,22.903,23.563,24.419,25.847,26.557
Mango,1.321,1.569,2.820,4.517,4.857,6.444,7.384,8.872,9.320,9.504,11.968,11.859,12.780,13.863,15.114,15.000,16.113,17.540,19.753,20.168,20.796,22.259,22.493,23.092,25.592,26.012
Blueberry,0.033,2.612,2.640,4.051,4.572,5.021,6.013,7.865,9.495,9.123,11.929,12.757,13.879,14.849,14.414,15.099,17.352,17.451,18.993,20.406,21.961,22.356,22.762,23.155,24.799,25.172


In [26]:
pretty_pandas(
    test_df, index='show', font_size=11, header_size=12, mymax=100,
    # fill_palette=['#FFFFDD','#DAECB8','#87C6BD','#4B96BE','#2E4C9B','#0D1D55'],
    fill_palette=['#e8f6b1', '#b2e1b6', '#65c3bf', '#2ca1c2', '#216daf', '#253997','#000'],
    rows = list(test_df.index)[8:18], #['Starfruit','Plum','Banana','Raspberry'],
    columns = ['B','C','D','E','F','G','H','I'],
    default_fill_color = '#F9F9F9',
    default_text_color = '#DDDDE4',
)

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z
Apple,1.322,1.637,3.672,3.688,5.272,5.750,7.064,8.777,9.274,10.732,11.873,12.679,12.424,13.487,14.277,15.436,16.074,18.944,18.129,20.546,20.894,22.147,22.785,24.366,24.497,26.824
Watermelon,0.681,2.885,3.863,4.230,4.710,6.214,7.784,8.963,9.173,10.464,11.931,12.225,12.353,14.942,14.930,15.897,16.566,18.433,19.911,19.974,20.694,21.521,22.071,23.821,24.136,26.591
Orange,1.242,2.314,3.936,4.028,4.849,6.944,6.956,7.332,9.971,9.311,10.397,12.182,12.043,14.284,14.191,15.858,17.744,18.702,18.951,19.080,20.718,21.468,22.075,24.437,25.763,26.952
Pear,0.986,1.171,3.159,4.820,5.849,6.030,6.239,8.239,8.447,10.519,10.541,11.342,13.057,13.647,15.201,15.356,16.774,17.345,18.670,20.224,20.044,22.628,23.433,23.923,25.307,26.027
Cherry,1.924,1.361,2.048,3.689,5.707,6.540,6.722,8.137,9.772,9.836,11.821,11.095,13.525,13.951,15.742,16.395,17.629,17.478,19.367,19.734,21.380,22.088,22.664,24.296,24.482,25.108
Strawberry,0.617,2.142,2.982,4.194,4.495,6.697,7.243,7.118,8.407,10.260,11.691,12.120,12.932,14.655,14.715,15.897,16.983,18.431,19.294,19.023,20.267,21.907,22.302,23.359,25.474,25.886
Nectarine,1.989,1.451,2.180,4.265,4.319,5.025,7.734,8.041,9.729,9.408,10.612,12.553,13.633,14.161,14.978,16.980,16.386,18.994,18.460,20.574,20.654,22.548,22.595,24.533,24.030,25.555
Grape,0.689,2.889,3.051,3.277,4.792,6.272,7.368,7.709,8.084,10.763,11.474,11.282,13.442,14.644,15.638,15.118,16.074,17.463,19.343,19.159,20.258,22.903,23.563,24.419,25.847,26.557
Mango,1.321,1.569,2.820,4.517,4.857,6.444,7.384,8.872,9.320,9.504,11.968,11.859,12.780,13.863,15.114,15.000,16.113,17.540,19.753,20.168,20.796,22.259,22.493,23.092,25.592,26.012
Blueberry,0.033,2.612,2.640,4.051,4.572,5.021,6.013,7.865,9.495,9.123,11.929,12.757,13.879,14.849,14.414,15.099,17.352,17.451,18.993,20.406,21.961,22.356,22.762,23.155,24.799,25.172


In [27]:
pretty_pandas(
    test_df, index='show', font_size=11, header_size=12, mymin=4, mymax=18,
    fill_palette=[list(sns.color_palette('YlOrRd').as_hex()),[]],
    default_fill_color = '#F9F9F9',
    default_text_color = '#555',
)

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z
Apple,1.322,1.637,3.672,3.688,5.272,5.750,7.064,8.777,9.274,10.732,11.873,12.679,12.424,13.487,14.277,15.436,16.074,18.944,18.129,20.546,20.894,22.147,22.785,24.366,24.497,26.824
Watermelon,0.681,2.885,3.863,4.230,4.710,6.214,7.784,8.963,9.173,10.464,11.931,12.225,12.353,14.942,14.930,15.897,16.566,18.433,19.911,19.974,20.694,21.521,22.071,23.821,24.136,26.591
Orange,1.242,2.314,3.936,4.028,4.849,6.944,6.956,7.332,9.971,9.311,10.397,12.182,12.043,14.284,14.191,15.858,17.744,18.702,18.951,19.080,20.718,21.468,22.075,24.437,25.763,26.952
Pear,0.986,1.171,3.159,4.820,5.849,6.030,6.239,8.239,8.447,10.519,10.541,11.342,13.057,13.647,15.201,15.356,16.774,17.345,18.670,20.224,20.044,22.628,23.433,23.923,25.307,26.027
Cherry,1.924,1.361,2.048,3.689,5.707,6.540,6.722,8.137,9.772,9.836,11.821,11.095,13.525,13.951,15.742,16.395,17.629,17.478,19.367,19.734,21.380,22.088,22.664,24.296,24.482,25.108
Strawberry,0.617,2.142,2.982,4.194,4.495,6.697,7.243,7.118,8.407,10.260,11.691,12.120,12.932,14.655,14.715,15.897,16.983,18.431,19.294,19.023,20.267,21.907,22.302,23.359,25.474,25.886
Nectarine,1.989,1.451,2.180,4.265,4.319,5.025,7.734,8.041,9.729,9.408,10.612,12.553,13.633,14.161,14.978,16.980,16.386,18.994,18.460,20.574,20.654,22.548,22.595,24.533,24.030,25.555
Grape,0.689,2.889,3.051,3.277,4.792,6.272,7.368,7.709,8.084,10.763,11.474,11.282,13.442,14.644,15.638,15.118,16.074,17.463,19.343,19.159,20.258,22.903,23.563,24.419,25.847,26.557
Mango,1.321,1.569,2.820,4.517,4.857,6.444,7.384,8.872,9.320,9.504,11.968,11.859,12.780,13.863,15.114,15.000,16.113,17.540,19.753,20.168,20.796,22.259,22.493,23.092,25.592,26.012
Blueberry,0.033,2.612,2.640,4.051,4.572,5.021,6.013,7.865,9.495,9.123,11.929,12.757,13.879,14.849,14.414,15.099,17.352,17.451,18.993,20.406,21.961,22.356,22.762,23.155,24.799,25.172


####
#### **MAIN STYLING FUNCTIONS - V2**

In [150]:
def apply_colors(col, value_transform=None, default_fill_color='#FFF', default_text_color='#000', default_border='', default_fill_text_colors=['#000','#FFF'],
                 thresholds=None, fill_palette=None, text_palette=None, rows=None, columns=None, mymin=None, mymax=None):

    if value_transform is not None:
        col = run_transform(col,value_transform)

    fill_palette = [None if f is None else list(get_rgb_colors(p) for p in f) for f in fill_palette]
    text_palette = [None if t is None else list(get_rgb_colors(p) for p in t) for t in text_palette]
    rgb_fill_vals,rgb_text_vals,fill_styles,default_text_styles,active_text_styles = [],[],[],[],[]
    default = 'background-color: '+default_fill_color+'; color: '+default_text_color+'; border: '+default_border
    styles = [default for j in range(len(col.values))]
    text_styles = ['' for j in range(len(col.values))]
    
    for i in range(len(fill_palette)):
        if fill_palette[i] is not None:
            if len(fill_palette[i]) == 1: # if the palette length is just 1 we just apply it globally - the trivial case
                rgb_fill_vals += [[fill_palette[i][0] for c in col.values]]
            else: # if the palette length is greater than 1, we assign each value a bespoke color based on its position in the full range
                fill_thresholds = divide_range(mymin[i], mymax[i], len(fill_palette[i]), thresholds)
                fill_quantiles = make_quantiles(col.values, fill_palette[i], mymin[i], mymax[i])
                rgb_fill_vals += [[generate_color(c, fill_thresholds[q:q+2], fill_palette[i][q:q+2]) for c,q in zip(col.values, fill_quantiles)]]
        else:
            rgb_fill_vals += [[None for c in col.values]]

        if text_palette[i] is not None:
            if len(text_palette[i]) == 1:
                rgb_text_vals += [[text_palette[i][0] for c in col.values]]
            else:
                text_thresholds = divide_range(mymin[i], mymax[i], len(text_palette[i]), thresholds)
                text_quantiles = make_quantiles(col.values, text_palette[i], mymin[i], mymax[i])
                rgb_text_vals += [[generate_color(c, text_thresholds[q:q+2], text_palette[i][q:q+2]) for c,q in zip(col.values, text_quantiles)]]
        else:
            rgb_text_vals += [[None for c in col.values]]

        fill_styles += [['background-color: '+('' if fill_palette[i] is None else '#'+''.join([make_hex_color(c) for c in v])) for v in rgb_fill_vals[-1]]]
        default_text_styles += [['color: '+('' if fill_palette[i] is None else (default_fill_text_colors[0] if luminosity(v)>=100 else default_fill_text_colors[1])) for v in rgb_fill_vals[-1]]]
        text_styles = ['color: #'+(text_styles[j] if text_palette[i] is None else ''.join([make_hex_color(c) for c in rgb_text_vals[-1][j]])) for j in range(len(col.values))]

        styles = ['; '.join([fill_styles[i][j],default_text_styles[i][j],text_styles[j]]) if (mymin[i] <= col.values[j] <= mymax[i]) and
                  (col.name in columns[i]) and (j in rows[i]) else styles[j] for j in range(len(col.values))]
    return styles

In [152]:
def pretty_pandas(df, fill_palette=None, text_palette=None, rows=None, columns=None, index='show', group=None, font_size=None,
                  thresholds=None, header_size=None, default_fill_color='#FFF', default_text_color='#000', default_border='',
                  default_fill_text_colors=['#000','#FFF'], bg='white', mymin=None, mymax=None, value_transform=None, configs=None):
    """Generate efficient dataframe styling with fully customizable inputs.
    Keyword arguments:

    todo
    """
    sdf = df.style
    rows_all,columns_all = list(df.index),list(df.columns)

    def absent():
        return None
    if configs is not None:
        configs_default = [defaultdict(absent) for c in configs]
        for c in range(len(configs)):
            for i in configs[c].keys():
                configs_default[c][i] = configs[c][i]
        fill_palette = [c['fill_palette'] for c in configs_default]
        text_palette = [c['text_palette'] for c in configs_default]
        rows = [c['rows'] if c['rows'] is not None else rows_all for c in configs_default]
        row_indices = [list(rows_all.index(i) for i in r) for r in rows]
        columns = [c['columns'] if c['columns'] is not None else columns_all for c in configs_default]
        mymin, mymax = [],[]
        for i,c in enumerate(configs_default):
            # if value_transform is None:
            df_subset = df.loc[rows[i],columns[i]]
            if value_transform is not None:
                if type(value_transform) is not list: value_transform = [value_transform]
                df_subset = run_transform(df_subset,value_transform)
            mymin.append(type_format(df_subset.values,c['mymin'],c['number']) if c['mymin'] is not None else np.min(df_subset.values))
            mymax.append(type_format(df_subset.values,c['mymax'],c['number']) if c['mymax'] is not None else np.max(df_subset.values))
            # return df_subset,mymin,mymax
        # mymax = [max(mymin[m]+1,mymax[m]) for m in range(len(mymax))] # to prevent any divide by zero later on

    else:
        if mymin is None: mymin=np.min(df.values)
        if mymax is None: mymax=np.max(df.values)
        if rows is None:
            rows = rows_all
        row_indices = [rows.index(r) for r in rows] #[list(rows.index(i) for i in r) for r in rows]
        if columns is None: columns = columns_all
        mymin,mymax,rows,columns = [mymin],[mymax],[rows],[columns]

    if index=='hide': sdf.hide_index()
    if header_size is None: header_size=font_size

    sdf.apply(apply_colors, value_transform=value_transform, default_fill_color=default_fill_color, default_text_color=default_text_color,
              default_fill_text_colors=default_fill_text_colors, thresholds=thresholds, default_border=default_border,
              fill_palette=fill_palette, text_palette=text_palette, rows=row_indices, columns=columns, mymin=mymin, mymax=mymax, axis=0)

    return sdf.format('{:.3f}').set_table_styles([{'selector':'tr','props':[('background-color',bg+' !important')]}])

    # sdf.set_properties(**{'font-size': str(font_size)+'pt'})
    # .set_table_styles([{'selector': 'th', 'props': [('font-size', str(22)+'pt !important')]}])

In [21]:
def make_palette(*args,number='pct',palette=['white','red','yellow','green','blue'],**kwargs):
    configs = []
    if len(args)==2 or len(args)>=len(palette):
        configs.extend([
            {'fill_palette': palette[:1], 'mymax': args[0], 'number': number},
            {'fill_palette': palette[-1:], 'mymin': args[-1], 'number': number}
        ])
    else:
        if number=='pct':
            args = [0]+list(args)+[100]
        else:
            args = [-np.inf]+list(args)+[np.inf]
    for i in range(len(args)-1):
        configs.append(
            {
                'fill_palette': palette if len(args)==2 else palette[i:i+2],
                #'text_palette': palette if len(args)==2 else palette[i:i+2],
                'mymin': args[i],
                'mymax': args[i+1],
                'number': number
            }
        )
    for c in configs:
        for k in kwargs.keys(): c[k]=kwargs[k]
    return configs

####
#### **EXAMPLES**
####
#### CUSTOM THRESHOLDS SPREAD OVER A WHOLE TABLE USING PERCENTAGE THRESHOLDS

In [24]:
pretty_pandas(
    test_df,
    index='show',
    font_size=11,
    header_size=12,
    default_fill_color='#F9F9FF',
    default_text_color='#555',
    #default_border='1px solid #CCC',
    configs=make_palette(5,80,palette=['#000','red','yellow','white'],number='pct') #,columns=['A','B','C','D','E','F','G','H'])
)

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z
Apple,0.143,2.828,3.257,3.121,5.600,6.726,6.166,7.851,8.047,9.058,11.124,12.193,12.384,13.273,14.240,16.388,17.855,18.322,18.725,19.329,20.515,21.429,23.300,23.976,24.527,25.826
Watermelon,1.487,1.540,2.982,4.639,5.979,6.365,6.800,8.099,8.859,10.451,10.968,11.752,13.517,14.854,15.970,15.221,17.702,18.853,19.472,20.972,21.430,22.633,22.834,24.973,24.097,26.254
Orange,1.632,2.203,2.602,3.735,5.868,5.268,6.601,8.363,8.449,10.533,11.643,11.659,12.396,13.774,15.042,15.048,17.891,18.850,19.634,19.568,21.377,22.121,22.812,24.742,24.195,26.578
Pear,1.474,1.644,3.990,4.194,5.544,5.309,6.457,8.402,9.354,10.126,10.335,11.495,13.292,13.317,15.344,16.478,16.342,18.806,18.784,19.619,21.598,22.678,22.129,24.789,24.073,25.952
Cherry,1.934,1.424,3.895,4.847,5.283,5.076,6.633,8.254,8.750,10.917,11.037,12.449,13.448,13.821,15.520,15.633,16.646,17.305,19.743,19.328,21.676,21.866,22.645,23.248,25.621,26.657
Strawberry,1.467,1.632,3.146,3.994,4.220,5.625,6.618,8.809,8.645,9.924,10.467,12.534,12.871,13.402,14.216,15.902,17.424,18.186,19.856,20.867,22.000,21.749,23.523,23.649,25.251,26.476
Nectarine,0.007,2.833,3.399,4.349,4.600,5.077,7.833,8.272,8.183,9.624,11.034,11.883,12.202,13.175,14.626,15.267,17.682,17.974,18.162,20.900,20.842,22.130,23.443,23.615,24.768,26.023
Grape,0.296,2.539,2.460,3.174,5.107,5.331,6.555,7.090,9.754,10.617,11.771,11.247,12.600,14.132,15.327,16.001,17.521,18.792,18.409,19.384,20.457,21.402,23.749,24.533,24.653,26.886
Mango,0.672,1.354,3.137,4.573,5.230,6.592,7.820,7.743,8.985,9.582,10.699,11.083,13.869,14.687,14.010,16.303,17.719,17.576,18.332,19.519,21.414,22.816,23.081,24.569,25.074,26.672
Blueberry,1.748,2.850,2.570,4.073,5.665,5.545,6.048,7.108,9.779,10.657,10.816,12.926,12.332,13.053,15.255,16.508,16.752,17.367,18.006,19.585,21.093,22.539,23.328,24.395,24.253,25.820


####
#### HOMING INTO A SPECIFIC RANGE

In [28]:
pretty_pandas(
    test_df, index='show', font_size=11, header_size=12, mymax=100,
    default_fill_color = '#F9F9F9',
    default_text_color = '#DDDDE4',
    configs=make_palette(0,100,
                         palette=['#e8f6b1', '#b2e1b6', '#65c3bf', '#2ca1c2', '#216daf', '#253997','#000'],
                         columns = ['B','C','D','E','F','G','H','I','M','R'],
                         rows = ['Orange','Grape','Starfruit','Plum','Banana','Raspberry'],
                         number='pct')
)

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z
Apple,0.143,2.828,3.257,3.121,5.600,6.726,6.166,7.851,8.047,9.058,11.124,12.193,12.384,13.273,14.240,16.388,17.855,18.322,18.725,19.329,20.515,21.429,23.300,23.976,24.527,25.826
Watermelon,1.487,1.540,2.982,4.639,5.979,6.365,6.800,8.099,8.859,10.451,10.968,11.752,13.517,14.854,15.970,15.221,17.702,18.853,19.472,20.972,21.430,22.633,22.834,24.973,24.097,26.254
Orange,1.632,2.203,2.602,3.735,5.868,5.268,6.601,8.363,8.449,10.533,11.643,11.659,12.396,13.774,15.042,15.048,17.891,18.850,19.634,19.568,21.377,22.121,22.812,24.742,24.195,26.578
Pear,1.474,1.644,3.990,4.194,5.544,5.309,6.457,8.402,9.354,10.126,10.335,11.495,13.292,13.317,15.344,16.478,16.342,18.806,18.784,19.619,21.598,22.678,22.129,24.789,24.073,25.952
Cherry,1.934,1.424,3.895,4.847,5.283,5.076,6.633,8.254,8.750,10.917,11.037,12.449,13.448,13.821,15.520,15.633,16.646,17.305,19.743,19.328,21.676,21.866,22.645,23.248,25.621,26.657
Strawberry,1.467,1.632,3.146,3.994,4.220,5.625,6.618,8.809,8.645,9.924,10.467,12.534,12.871,13.402,14.216,15.902,17.424,18.186,19.856,20.867,22.000,21.749,23.523,23.649,25.251,26.476
Nectarine,0.007,2.833,3.399,4.349,4.600,5.077,7.833,8.272,8.183,9.624,11.034,11.883,12.202,13.175,14.626,15.267,17.682,17.974,18.162,20.900,20.842,22.130,23.443,23.615,24.768,26.023
Grape,0.296,2.539,2.460,3.174,5.107,5.331,6.555,7.090,9.754,10.617,11.771,11.247,12.600,14.132,15.327,16.001,17.521,18.792,18.409,19.384,20.457,21.402,23.749,24.533,24.653,26.886
Mango,0.672,1.354,3.137,4.573,5.230,6.592,7.820,7.743,8.985,9.582,10.699,11.083,13.869,14.687,14.010,16.303,17.719,17.576,18.332,19.519,21.414,22.816,23.081,24.569,25.074,26.672
Blueberry,1.748,2.850,2.570,4.073,5.665,5.545,6.048,7.108,9.779,10.657,10.816,12.926,12.332,13.053,15.255,16.508,16.752,17.367,18.006,19.585,21.093,22.539,23.328,24.395,24.253,25.820


#### 
#### HIGHLIGHTING POSITIVE AND NEGATIVE VALUES

In [29]:
pos_neg_fill=[
    {
        'mymin': 0,
        'fill_palette': ['green']
    },
    {
        'mymax': 0,
        'fill_palette': ['red']
    },
    {
        'mymin': -1,
        'mymax': 1,
        'fill_palette': ['white']
    }
]

pretty_pandas(
    test_df-5.849,
    configs=pos_neg_fill
)

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z
Apple,-5.706,-3.021,-2.592,-2.728,-0.249,0.877,0.317,2.002,2.198,3.209,5.275,6.344,6.535,7.424,8.391,10.539,12.006,12.473,12.876,13.480,14.666,15.580,17.451,18.127,18.678,19.977
Watermelon,-4.362,-4.309,-2.867,-1.210,0.130,0.516,0.951,2.250,3.010,4.602,5.119,5.903,7.668,9.005,10.121,9.372,11.853,13.004,13.623,15.123,15.581,16.784,16.985,19.124,18.248,20.405
Orange,-4.217,-3.646,-3.247,-2.114,0.019,-0.581,0.752,2.514,2.600,4.684,5.794,5.810,6.547,7.925,9.193,9.199,12.042,13.001,13.785,13.719,15.528,16.272,16.963,18.893,18.346,20.729
Pear,-4.375,-4.205,-1.859,-1.655,-0.305,-0.540,0.608,2.553,3.505,4.277,4.486,5.646,7.443,7.468,9.495,10.629,10.493,12.957,12.935,13.770,15.749,16.829,16.280,18.940,18.224,20.103
Cherry,-3.915,-4.425,-1.954,-1.002,-0.566,-0.773,0.784,2.405,2.901,5.068,5.188,6.600,7.599,7.972,9.671,9.784,10.797,11.456,13.894,13.479,15.827,16.017,16.796,17.399,19.772,20.808
Strawberry,-4.382,-4.217,-2.703,-1.855,-1.629,-0.224,0.769,2.960,2.796,4.075,4.618,6.685,7.022,7.553,8.367,10.053,11.575,12.337,14.007,15.018,16.151,15.900,17.674,17.800,19.402,20.627
Nectarine,-5.842,-3.016,-2.450,-1.500,-1.249,-0.772,1.984,2.423,2.334,3.775,5.185,6.034,6.353,7.326,8.777,9.418,11.833,12.125,12.313,15.051,14.993,16.281,17.594,17.766,18.919,20.174
Grape,-5.553,-3.310,-3.389,-2.675,-0.742,-0.518,0.706,1.241,3.905,4.768,5.922,5.398,6.751,8.283,9.478,10.152,11.672,12.943,12.560,13.535,14.608,15.553,17.900,18.684,18.804,21.037
Mango,-5.177,-4.495,-2.712,-1.276,-0.619,0.743,1.971,1.894,3.136,3.733,4.850,5.234,8.020,8.838,8.161,10.454,11.870,11.727,12.483,13.670,15.565,16.967,17.232,18.720,19.225,20.823
Blueberry,-4.101,-2.999,-3.279,-1.776,-0.184,-0.304,0.199,1.259,3.930,4.808,4.967,7.077,6.483,7.204,9.406,10.659,10.903,11.518,12.157,13.736,15.244,16.690,17.479,18.546,18.404,19.971


#### CUSTOM COLUMNS

In [30]:
configs=[
    {
        'fill_palette': ['#DDE7F7'], #list(sns.color_palette('YlOrRd').as_hex()),
        'columns': [test_df.columns[i] for i in range(0,len(test_df.columns),3)]
    }
]

pretty_pandas(
    test_df+5,
    index='show',
    font_size=11,
    header_size=12,
    default_fill_color='#F9F9FF',
    default_text_color='#555',
    configs=configs
)

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z
Apple,5.143,7.828,8.257,8.121,10.600,11.726,11.166,12.851,13.047,14.058,16.124,17.193,17.384,18.273,19.240,21.388,22.855,23.322,23.725,24.329,25.515,26.429,28.300,28.976,29.527,30.826
Watermelon,6.487,6.540,7.982,9.639,10.979,11.365,11.800,13.099,13.859,15.451,15.968,16.752,18.517,19.854,20.970,20.221,22.702,23.853,24.472,25.972,26.430,27.633,27.834,29.973,29.097,31.254
Orange,6.632,7.203,7.602,8.735,10.868,10.268,11.601,13.363,13.449,15.533,16.643,16.659,17.396,18.774,20.042,20.048,22.891,23.850,24.634,24.568,26.377,27.121,27.812,29.742,29.195,31.578
Pear,6.474,6.644,8.990,9.194,10.544,10.309,11.457,13.402,14.354,15.126,15.335,16.495,18.292,18.317,20.344,21.478,21.342,23.806,23.784,24.619,26.598,27.678,27.129,29.789,29.073,30.952
Cherry,6.934,6.424,8.895,9.847,10.283,10.076,11.633,13.254,13.750,15.917,16.037,17.449,18.448,18.821,20.520,20.633,21.646,22.305,24.743,24.328,26.676,26.866,27.645,28.248,30.621,31.657
Strawberry,6.467,6.632,8.146,8.994,9.220,10.625,11.618,13.809,13.645,14.924,15.467,17.534,17.871,18.402,19.216,20.902,22.424,23.186,24.856,25.867,27.000,26.749,28.523,28.649,30.251,31.476
Nectarine,5.007,7.833,8.399,9.349,9.600,10.077,12.833,13.272,13.183,14.624,16.034,16.883,17.202,18.175,19.626,20.267,22.682,22.974,23.162,25.900,25.842,27.130,28.443,28.615,29.768,31.023
Grape,5.296,7.539,7.460,8.174,10.107,10.331,11.555,12.090,14.754,15.617,16.771,16.247,17.600,19.132,20.327,21.001,22.521,23.792,23.409,24.384,25.457,26.402,28.749,29.533,29.653,31.886
Mango,5.672,6.354,8.137,9.573,10.230,11.592,12.820,12.743,13.985,14.582,15.699,16.083,18.869,19.687,19.010,21.303,22.719,22.576,23.332,24.519,26.414,27.816,28.081,29.569,30.074,31.672
Blueberry,6.748,7.850,7.570,9.073,10.665,10.545,11.048,12.108,14.779,15.657,15.816,17.926,17.332,18.053,20.255,21.508,21.752,22.367,23.006,24.585,26.093,27.539,28.328,29.395,29.253,30.820


#### 
#### CUSTOM ROWS

In [64]:
configs=[
    {
        'fill_palette': ['#DDE7F7'], #list(sns.color_palette('YlOrRd').as_hex()),
        'rows': [test_df.index[i] for i in range(0,len(test_df.index),7)]
    }
]

pretty_pandas(
    test_df+5,
    index='show',
    font_size=11,
    header_size=12,
    default_fill_color='#F9F9FF',
    default_text_color='#555',
    configs=configs
)

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z
Apple,5.143,7.828,8.257,8.121,10.600,11.726,11.166,12.851,13.047,14.058,16.124,17.193,17.384,18.273,19.240,21.388,22.855,23.322,23.725,24.329,25.515,26.429,28.300,28.976,29.527,30.826
Watermelon,6.487,6.540,7.982,9.639,10.979,11.365,11.800,13.099,13.859,15.451,15.968,16.752,18.517,19.854,20.970,20.221,22.702,23.853,24.472,25.972,26.430,27.633,27.834,29.973,29.097,31.254
Orange,6.632,7.203,7.602,8.735,10.868,10.268,11.601,13.363,13.449,15.533,16.643,16.659,17.396,18.774,20.042,20.048,22.891,23.850,24.634,24.568,26.377,27.121,27.812,29.742,29.195,31.578
Pear,6.474,6.644,8.990,9.194,10.544,10.309,11.457,13.402,14.354,15.126,15.335,16.495,18.292,18.317,20.344,21.478,21.342,23.806,23.784,24.619,26.598,27.678,27.129,29.789,29.073,30.952
Cherry,6.934,6.424,8.895,9.847,10.283,10.076,11.633,13.254,13.750,15.917,16.037,17.449,18.448,18.821,20.520,20.633,21.646,22.305,24.743,24.328,26.676,26.866,27.645,28.248,30.621,31.657
Strawberry,6.467,6.632,8.146,8.994,9.220,10.625,11.618,13.809,13.645,14.924,15.467,17.534,17.871,18.402,19.216,20.902,22.424,23.186,24.856,25.867,27.000,26.749,28.523,28.649,30.251,31.476
Nectarine,5.007,7.833,8.399,9.349,9.600,10.077,12.833,13.272,13.183,14.624,16.034,16.883,17.202,18.175,19.626,20.267,22.682,22.974,23.162,25.900,25.842,27.130,28.443,28.615,29.768,31.023
Grape,5.296,7.539,7.460,8.174,10.107,10.331,11.555,12.090,14.754,15.617,16.771,16.247,17.600,19.132,20.327,21.001,22.521,23.792,23.409,24.384,25.457,26.402,28.749,29.533,29.653,31.886
Mango,5.672,6.354,8.137,9.573,10.230,11.592,12.820,12.743,13.985,14.582,15.699,16.083,18.869,19.687,19.010,21.303,22.719,22.576,23.332,24.519,26.414,27.816,28.081,29.569,30.074,31.672
Blueberry,6.748,7.850,7.570,9.073,10.665,10.545,11.048,12.108,14.779,15.657,15.816,17.926,17.332,18.053,20.255,21.508,21.752,22.367,23.006,24.585,26.093,27.539,28.328,29.395,29.253,30.820


#### 
#### NONLINEAR SCALE

In [158]:
configs=[
    {
        'fill_palette': ['#FFF','#DDE7F7','#004']
    }
]

def squares(col):
    return np.power(col,2)

pretty_pandas(
    test_df,
    value_transform=[squares], #squares, # can also use np.lop, np.exp, for example.
    index='show',
    font_size=11,
    header_size=12,
    default_fill_color='#F9F9FF',
    default_text_color='#555',
    configs=configs
)

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z
Apple,0.143,2.828,3.257,3.121,5.600,6.726,6.166,7.851,8.047,9.058,11.124,12.193,12.384,13.273,14.240,16.388,17.855,18.322,18.725,19.329,20.515,21.429,23.300,23.976,24.527,25.826
Watermelon,1.487,1.540,2.982,4.639,5.979,6.365,6.800,8.099,8.859,10.451,10.968,11.752,13.517,14.854,15.970,15.221,17.702,18.853,19.472,20.972,21.430,22.633,22.834,24.973,24.097,26.254
Orange,1.632,2.203,2.602,3.735,5.868,5.268,6.601,8.363,8.449,10.533,11.643,11.659,12.396,13.774,15.042,15.048,17.891,18.850,19.634,19.568,21.377,22.121,22.812,24.742,24.195,26.578
Pear,1.474,1.644,3.990,4.194,5.544,5.309,6.457,8.402,9.354,10.126,10.335,11.495,13.292,13.317,15.344,16.478,16.342,18.806,18.784,19.619,21.598,22.678,22.129,24.789,24.073,25.952
Cherry,1.934,1.424,3.895,4.847,5.283,5.076,6.633,8.254,8.750,10.917,11.037,12.449,13.448,13.821,15.520,15.633,16.646,17.305,19.743,19.328,21.676,21.866,22.645,23.248,25.621,26.657
Strawberry,1.467,1.632,3.146,3.994,4.220,5.625,6.618,8.809,8.645,9.924,10.467,12.534,12.871,13.402,14.216,15.902,17.424,18.186,19.856,20.867,22.000,21.749,23.523,23.649,25.251,26.476
Nectarine,0.007,2.833,3.399,4.349,4.600,5.077,7.833,8.272,8.183,9.624,11.034,11.883,12.202,13.175,14.626,15.267,17.682,17.974,18.162,20.900,20.842,22.130,23.443,23.615,24.768,26.023
Grape,0.296,2.539,2.460,3.174,5.107,5.331,6.555,7.090,9.754,10.617,11.771,11.247,12.600,14.132,15.327,16.001,17.521,18.792,18.409,19.384,20.457,21.402,23.749,24.533,24.653,26.886
Mango,0.672,1.354,3.137,4.573,5.230,6.592,7.820,7.743,8.985,9.582,10.699,11.083,13.869,14.687,14.010,16.303,17.719,17.576,18.332,19.519,21.414,22.816,23.081,24.569,25.074,26.672
Blueberry,1.748,2.850,2.570,4.073,5.665,5.545,6.048,7.108,9.779,10.657,10.816,12.926,12.332,13.053,15.255,16.508,16.752,17.367,18.006,19.585,21.093,22.539,23.328,24.395,24.253,25.820
